In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing import timeseries_dataset_from_array

In [2]:
krakow_20_23 = pd.read_csv('krakow_data_20-23_preprocessed.csv')

start_date = '2020-01-01 00:00:00'
pivot_date = '2022-05-01 00:00:00'
stop_date = '2022-12-31 23:00:00'
krakow_20_23.set_index('timestamp', inplace=True)

In [3]:
predict_params = [
    'Wysokość podstawy niższej  [m]',
    'Wysokość podstawy wyższej [m]',
    'Widzialność  [kod]',
    'Widzialność operatora [m]',
    'Zachmurzenie ogólne [oktanty]',
    'Kierunek wiatru  [°]',
    'Prędkość wiatru  [m/s]',
    'Poryw wiatru  [m/s]',
    'Temperatura powietrza [°C]/1',
    'Ciśnienie pary wodnej [hPa]',
    'Wilgotność względna [%]',
    'Temperatura punktu rosy [°C]',
    'Ciśnienie na pozimie stacji [hPa]',
    'Ciśnienie na pozimie morza [hPa]',
    'Charakterystyka tendencji [kod]',
    'Wartość tendencji [wartość]',
    'Opad za 6 godzin [mm]',
    'Rodzaj opadu za 6 godzin [kod]',
    'Pogoda ubiegła [kod]',
    'Zachmurzenie niskie [oktanty]',
    'Stan gruntu [kod]',
    'Niedosyt wilgotności [hPa]',
    'Wystąpienie rosy [0/1]',
    'Poryw maksymalny za okres WW [m/s]',
    'Temperatura minimalna za 12 godzin  [°C]',
    'Temperatura maksymalna za 12 godzin [°C]',
    'Równoważnik wodny śniegu [mm/cm]',
    'Wysokość pokrywy śnieżnej [cm]',
    'Wysokość świeżo spadłego śniegu  [cm]',
    'Wysokość śniegu na poletku [cm]',
    'Wysokość próbki [cm]',
    'Ciężar próbki [g]',
    'Pogoda bieżąca [kod]'
]


In [4]:
train_data = krakow_20_23[start_date: pivot_date]
test_data = krakow_20_23[pivot_date: stop_date]

In [5]:
krakow_20_23 = krakow_20_23[predict_params]

In [6]:
window_size = 24 * 2
batch_size = 30

input_data = []
target = []
for i in range(len(train_X) - window_size):
    input_data.append(train_X[i:i + window_size])  # Use past 'window_size' values as input
    target.append(train_Y[i + window_size])  # Predict the last column ('target') for the next step
    
input_data = np.array(input_data)
target = np.array(target)
target.shape


NameError: name 'train_X' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(64, activation='relu', input_shape=(look_back, 32)))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mse', 'mae'])
    return model

In [ ]:
model=model_dnn(window_size)

history=model.fit(input_data,
                  target, 
                  epochs=100, 
                  batch_size=30, 
                  verbose=1, 
                  shuffle=False)

In [ ]:
window_size = 24 * 2
batch_size = 30

test_data = []

for i in range(len(test_X) - window_size):
    test_data.append(test_X[i:i + window_size])  # Use past 'window_size' values as input
    
test_data = np.array(test_data)

In [ ]:
test_data.shape

In [ ]:
predictions = model.predict(test_data)
predictions.shape

In [ ]:
predictions

In [11]:
import numpy as np
import pandas as pd
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming you have your dataset loaded into a pandas DataFrame named 'data'
# 'data' should have 35 predictor columns (X) and 1 output column (y)

# Assuming the predictors are in columns 0 to 34 and the output is in column 35
# Modify this according to your actual dataset

# Function to create time series dataset with specified window size
def create_time_series_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size), :-1])
        y.append(data[i + window_size, -1])
    return np.array(X), np.array(y)

# Convert dataframe to numpy array
dataset = krakow_20_23.values

# Normalize the dataset
scaler = MinMaxScaler()
dataset_normalized = scaler.fit_transform(dataset)

# Create time series dataset with window size 48
window_size = 48
X, y = create_time_series_dataset(dataset_normalized, window_size)

# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build the DNN model
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(window_size, 32)))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
model.add(Dense(units=1))  # Output layer with 1 neuron for the predicted output

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=1, batch_size=32)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

# Make predictions
predictions = model.predict(X_test)

# Inverse transform the predictions to original scale
predictions = scaler.inverse_transform(X_test + predictions)





165/165 [==============================] - 4s 18ms/step - loss: 0.0308
Test Loss: 0.03080487810075283
165/165 [==============================] - 4s 17ms/step


ValueError: operands could not be broadcast together with shapes (5252,48,32) (5252,1) 

In [21]:
predictions = model.predict(X_test)
predictions

165/165 [==============================] - 3s 20ms/step


array([[0.13509576],
       [0.14541577],
       [0.15133774],
       ...,
       [0.12215139],
       [0.11778735],
       [0.10858485]], dtype=float32)

In [27]:
merged = np.hstack((X_test[:, -1, :], predictions))

In [28]:
predictions = scaler.inverse_transform(merged)

In [29]:
predictions

array([[1800.        ,    0.        ,    8.        , ...,    0.        ,
           0.        ,   13.96919298],
       [   0.        ,    0.        ,    8.        , ...,    0.        ,
           0.        ,   14.95991373],
       [   0.        ,    0.        ,    8.        , ...,    0.        ,
           0.        ,   15.52842331],
       ...,
       [   0.        ,    0.        ,    8.        , ...,    0.        ,
           0.        ,   12.72653341],
       [   0.        ,    0.        ,    8.        , ...,    0.        ,
           0.        ,   12.30758524],
       [   0.        ,    0.        ,    8.        , ...,    0.        ,
           0.        ,   11.4241457 ]])